# 相関分析を実施するプログラム

In [1]:
import sys
sys.path.append('..')
from Constants import Constants

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import re

from sklearn.linear_model import LinearRegression
import seaborn

import pyperclip
import os

In [3]:
pd.set_option('display.max_columns', 50)

In [4]:
mkdir_path = f"{Constants.path_to_matlab_repository_folder}/csv/analysis_results/{Constants.folder_name}"

In [5]:
%pwd

'/Users/miyakooti/repositories/ECGAnalysis'

In [6]:
os.system("mkdir " + mkdir_path)
os.system("mkdir " + mkdir_path + "/correlation")

mkdir: /Users/miyakooti/repositories/arai_MATLAB_program/csv/analysis_results/20240917_kyoto: File exists
mkdir: /Users/miyakooti/repositories/arai_MATLAB_program/csv/analysis_results/20240917_kyoto/correlation: File exists


256

### 分析対象のファイルのパスを正規表現で取得

In [7]:
pathes = glob.glob(f"{Constants.path_to_matlab_repository_folder}/csv/*_*/HRV_and_PLI.csv")
pathes.sort()
pathes

['/Users/miyakooti/repositories/arai_MATLAB_program/csv/0_kumakura/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/10_tenshin/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/11_masanori/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/12_enoki/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/13_ito/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/14_yoshioka/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/15_chizuru/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/1_kim/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/2_souma/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/3_fujii/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/4_tubota/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/5_toki/HRV_and_PLI.csv',
 

In [8]:
from scipy.stats import pearsonr
import statistics

### 全被験者のデータをconcatする

In [9]:
#行の表示を省略しない
pd.set_option('display.max_rows', None)

#列の表示を省略しない
pd.set_option('display.max_columns', None)

In [10]:
Constants.target_phases

['boredom', 'flow', 'ultra', 'overload']

In [11]:
for i,target_phase in enumerate(Constants.target_phases):
    
    for j,path in enumerate(pathes):

        df = pd.read_csv(path,index_col=0).fillna(0)

        df = df.loc[[target_phase]] # seriesとして取り出したいときはこっち
        if j == 0 and i == 0:
            print("flow_datasetを初期化しました")
            flow_dataset = df
        else:
            flow_dataset = pd.concat([flow_dataset, df], axis=0)

flow_dataset

flow_datasetを初期化しました


,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate,vlf,lf,hf,lf/hf,p_total,vlf_perc,lf_perc,hf_perc,lf_nu,hf_nu,min_lf,min_hf,lf_diff,hf_diff,lfhf_diff,kubios_lf,kubios_hf,kubios_lf/hf,FpZ,FC3,FC4,FCz,O1,O2,Fp2,mean_all,mean_3ch,concentration_FC3,concentration_FC4,concentration_FCz,max_concentration_FC3,max_concentration_FC4,max_concentration_FCz,concentration_mean,max_concentration_mean,difficulty,fast,fulfillment,absorb,questionnaire_average
boredom,75.438926,795.345361,41.451926,17.859601,33.455281,0.609819,0.129199,24.0,23.656319,53.599844,3983.461053,0.441351,0.356460,231.553019,291.101937,336.954289,0.863921,859.609245,26.937009,33.864449,39.198542,46.349662,53.650338,93.234866,334.576916,-84.483790,144.126682,-0.434434,318.0,315.0,1.000,0.276021,0.446082,0.437729,0.560950,0.180391,0.162976,0.240775,0.327370,0.481587,0.272161,0.092863,0.222104,0.476316,0.418886,0.471890,0.195709,0.455698,2,2.0,5.0,2.0,2.75
boredom,88.817951,675.539112,40.040559,18.638951,30.130648,0.474576,0.101695,30.0,21.305037,52.333279,3502.757897,0.407103,0.125184,230.918805,688.767033,224.990607,3.061315,1144.676445,20.173282,60.171329,19.655389,75.377431,24.622569,478.893112,193.058120,-132.918143,-8.235106,-0.496910,604.0,229.0,2.639,0.448846,0.607136,0.642968,0.732918,0.286166,0.396024,0.466052,0.284452,0.661008,-0.013875,0.047647,-0.007030,0.166432,0.203619,0.118208,0.008914,0.162753,3,2.0,4.0,2.0,2.75
boredom,71.786562,835.811024,47.251002,15.327906,24.798981,0.400000,0.042105,28.0,17.533910,64.333409,3543.767459,0.272548,0.125612,701.772575,1266.585276,184.562388,6.862640,2152.920239,32.596311,58.831036,8.572653,87.281626,12.718374,633.640214,166.823132,-351.297468,-49.402896,-0.090731,963.0,202.0,4.770,0.348395,0.550590,0.565329,0.610800,0.308010,0.451657,0.362960,0.278507,0.575573,-0.025012,-0.260113,-0.123365,0.477440,0.243520,0.422430,-0.136163,0.381130,1,1.0,2.0,1.0,1.25
boredom,75.843941,791.098131,66.946482,14.202609,22.917426,0.353630,0.023419,49.0,16.202564,93.024553,4735.122491,0.174175,0.147671,1490.230066,802.798502,199.634560,4.021340,2492.663127,59.784656,32.206458,8.008886,80.084998,19.915002,621.964545,72.667348,3345.451546,-63.851212,26.529166,890.0,242.0,3.680,0.358975,0.495150,0.549294,0.599081,0.207888,0.253009,0.326009,0.259150,0.547842,0.024070,0.000098,0.097890,0.229920,0.227478,0.243923,0.040686,0.233774,2,1.0,4.0,5.0,3.00
boredom,70.209932,854.579946,75.423715,37.476253,61.627738,0.737705,0.409836,48.0,43.577227,97.343777,13326.547703,0.447663,0.253694,1272.262127,1203.311501,1633.576699,0.736612,4109.150328,30.961684,29.283706,39.754610,42.416599,57.583401,1203.034285,1626.589519,1141.928497,-263.918101,0.975669,1939.0,1974.0,0.982,0.381545,0.618994,0.686718,0.722451,0.375563,0.380402,0.303913,0.195918,0.676054,-0.181278,-0.192921,-0.238430,0.181357,0.097037,-0.063873,-0.204210,0.071507,1,3.0,4.0,4.0,3.00
boredom,74.841495,801.694301,33.826743,15.629643,28.536238,0.532468,0.057143,24.0,20.178034,43.418627,2752.357208,0.464732,0.226208,131.299495,262.146750,575.422986,0.455572,968.869231,13.551828,27.056980,59.391192,31.298498,68.701502,262.073456,574.915605,69.212411,-110.667445,0.257403,206.0,494.0,0.418,0.274297,0.312096,0.415012,0.637438,0.128878,0.155111,0.297436,0.190226,0.454849,-0.140224,0.004087,-0.084016,0.247489,0.190367,0.257137,-0.073384,0.231665,1,3.0,2.0,2.0,2.00
boredom,76.028596,789.176744,33.230715,10.524766,17.299501,0.202797,0.002331,16.0,12.232494,45.420276,1745.479158,0.269318,0.147345,980.050626,851.551300,153.977297,5.530369,1985.579223,49.358425,42.886795,7.754780,84.686930,15.313070,587.444519,98.512711,-732.431522,-6.723485,-4.721427,385.0,223.0,1.721,0.313061,0.346950,0.437815,0.602253,0.197531,0.168402,0.216386,0.237270,0.462339,-0.051074,0.127397,0.101445,0.135659,0.322124,0.321942,0.059256,0.259908,7,3.0,7.0,6.0,5.75
boredom,70.629782,849.500000,47.316105,16.830978,28.476891,0.504000,0.074667,31.0,20.135488,63.821153,4037.166609,0.315499,0.125231,426.247032,544.3

# difficultyを数値に変換する

In [12]:
flow_dataset["difficulty"] = flow_dataset["difficulty"].astype("float")

In [13]:
for survey_item in Constants.survey_items:
    

    # 保存用dict
    export_data = {    
        "target": Constants.target_columns,
        "mean": [],
        "SD": [],
        "R": [],
        "p": [],
    }

    # それぞれの説明変数に対して，，，
    for target_column in Constants.target_columns:
        print(f"{survey_item}に対する{target_column}の相関係数を計算します．")

        x = flow_dataset[target_column]
        y = flow_dataset[[survey_item]]

        # ピアソンの積率相関係数を計算        
        R, p = pearsonr(x, y)

        export_data["mean"].append(statistics.mean(x))
        export_data["SD"].append(statistics.pstdev(x))
        export_data["R"].append(R[0])
        export_data["p"].append(p)

    df = pd.DataFrame(export_data)

    # 保存
    save_path = f"{Constants.path_to_matlab_repository_folder}/csv/analysis_results/{Constants.folder_name}/correlation/flowscore_and_{survey_item}.csv"
    df.to_csv(save_path)  

difficultyに対するbpmの相関係数を計算します．
difficultyに対するibiの相関係数を計算します．
difficultyに対するlfの相関係数を計算します．
difficultyに対するhfの相関係数を計算します．
difficultyに対するmin_lfの相関係数を計算します．
difficultyに対するmin_hfの相関係数を計算します．
difficultyに対するlf/hfの相関係数を計算します．
difficultyに対するkubios_lfの相関係数を計算します．
difficultyに対するkubios_hfの相関係数を計算します．
difficultyに対するkubios_lf/hfの相関係数を計算します．
difficultyに対するsdsdの相関係数を計算します．
difficultyに対するrmssdの相関係数を計算します．
difficultyに対するpnn20の相関係数を計算します．
difficultyに対するpnn50の相関係数を計算します．
difficultyに対するconcentration_FC3の相関係数を計算します．
difficultyに対するconcentration_FC4の相関係数を計算します．
difficultyに対するconcentration_FCzの相関係数を計算します．
difficultyに対するconcentration_meanの相関係数を計算します．
difficultyに対するFC3の相関係数を計算します．
difficultyに対するFC4の相関係数を計算します．
difficultyに対するFCzの相関係数を計算します．
difficultyに対するmean_3chの相関係数を計算します．
difficultyに対するmax_concentration_FC3の相関係数を計算します．
difficultyに対するmax_concentration_FC4の相関係数を計算します．
difficultyに対するmax_concentration_FCzの相関係数を計算します．
difficultyに対するmax_concentration_meanの相関係数を計算します．
difficultyに対するlf_diffの相関係数を計算します．
difficultyに対するhf_

# 参考文献
- https://www.st-hakky-blog.com/entry/2018/01/30/004659